In [ ]:
#| default_exp util.parser

In [ ]:
#| export
import json
from pathlib import Path

In [ ]:
#| export
def find_source(
    filename: str,
    #   cell_type: str = None,
    tags: list,
) -> list:

    nb = json.load(open(filename,))
    cells = []
    for cell in nb['cells']:

        #         if 'source' not in cell.keys():
        #             continue
        if len(cell['source']) == 0:
            continue

        if cell['source'][0].strip() in tags:
            cell['source'] = cell['source'][1:]
            cells.append(cell)

    return cells

In [ ]:
find_source(
    './test_for_parser.json',
    tags=['##%% closing'],
);

In [ ]:
#| export
def generate_notebook(
    absolute_path: Path,
    filename: str,
):

    old_nb = json.load(open(filename,))

    new_nb = old_nb.copy()

    new_nb['cells'] = []

    for cell in old_nb['cells']:

        if len(cell['source']) == 0:
            new_nb['cells'].append(cell)
            continue
            
        if cell['source'][0].strip() == "##%% ommit":
            continue

        if cell['source'][0].strip() == "##%% import":

            fn = cell['source'][1].split('#')[-1].strip()
            fn = Path.joinpath(absolute_path, fn)
            new_sources = find_source(
                fn,
                tags=['##%% export'],
            )

            new_nb['cells'] = new_nb['cells'] + new_sources
        else:

            new_nb['cells'].append(cell)

    return new_nb

In [ ]:
#| export
def notebook_to_file(
    absolute_path: Path,
    source: str,
    target: str,
) -> None:
    new_nb = generate_notebook(
        absolute_path=absolute_path,
        filename=source,
    )
    json.dump(new_nb, open(target, 'w'))

In [ ]:
a = generate_notebook(
    absolute_path=Path.cwd().parent,
    filename=Path(Path.cwd().parent, '02_Template.ipynb'),
)

In [ ]:
#a['cells']

In [ ]:
#| export


def output_to_markdown(
    absolute_path: Path,
    source: str,
    target: str,
):
    """Extracts an standard output from the flag `##%% output`, converts it 
    to a markdown cell and appends it to the notebook.
    """

    old_nb = json.load(open(source,))

    new_nb = old_nb.copy()

    new_nb['cells'] = []

    for cell in old_nb['cells']:

        new_nb['cells'].append(cell)

        if len(cell['source']) == 0 or len(cell['outputs']) == 0:
            continue
            
        if cell['source'][0].strip() == "##%% ommit":
            continue

        if cell['source'][0].strip() == "##%% import":

            new_source = [
                ' '.join(output['text']) for output in cell['outputs']
            ]
            new_cell = cell.copy()

            new_cell['cell_type'] = 'markdown'
            new_cell['id'] = 'new-markdown-' + cell['id']
            new_cell['outputs'] = []
            new_cell['source'] = ["##%% export  "] + ["\n"] + new_source

            new_nb['cells'].append(new_cell,)

    json.dump(new_nb, open(target, 'w'))

In [ ]:
#| export


class Parser:

    def __init__(self):
        self.current_path = Path.cwd()

    @staticmethod
    def outputs_to_markdowns(filename: str) -> None:

        parser = Parser()

        output_to_markdown(
            absolute_path=parser.current_path,
            source=filename,
            target=filename,
        )